## Implement the A* search algorithm for a pathfinding problem.

In [ ]:
import tkinter as tk
from tkinter import simpledialog
from queue import PriorityQueue
import time
import itertools

CELL_SIZE = 30
DELAY = 50  

class Cell:
    def __init__(self, row, col, canvas):
        self.row = row
        self.col = col
        self.canvas = canvas
        self.state = "empty"
        self.rect = canvas.create_rectangle(
            col * CELL_SIZE, row * CELL_SIZE,
            (col + 1) * CELL_SIZE, (row + 1) * CELL_SIZE,
            fill="white", outline="gray"
        )

    def set_state(self, state):
        self.state = state
        color_map = {
            "empty": "white",
            "wall": "black",
            "start": "green",
            "end": "red",
            "visited": "lightblue",
            "path": "blue",
            "current": "yellow"
        }
        self.canvas.itemconfig(self.rect, fill=color_map[state])

class AStarVisualizer:
    def __init__(self, root, size):
        self.root = root
        self.size = size
        self.canvas = tk.Canvas(root, width=CELL_SIZE * size, height=CELL_SIZE * size)
        self.canvas.pack()

        self.grid = [[Cell(i, j, self.canvas) for j in range(size)] for i in range(size)]

        self.start = None
        self.end = None
        self.setting = "start"
        self.counter = itertools.count() 

        self.canvas.bind("<Button-1>", self.on_click)

        self.run_button = tk.Button(root, text="Run A*", command=self.run_astar)
        self.run_button.pack(pady=5)

    def on_click(self, event):
        row = event.y // CELL_SIZE
        col = event.x // CELL_SIZE
        cell = self.grid[row][col]

        if self.setting == "start":
            if self.start:
                self.start.set_state("empty")
            self.start = cell
            cell.set_state("start")
            self.setting = "end"

        elif self.setting == "end":
            if self.end:
                self.end.set_state("empty")
            self.end = cell
            cell.set_state("end")
            self.setting = "walls"

        elif self.setting == "walls":
            if cell.state == "wall":
                cell.set_state("empty")
            elif cell.state == "empty":
                cell.set_state("wall")

    def run_astar(self):
        if not self.start or not self.end:
            print("Start or End not set.")
            return

        open_set = PriorityQueue()
        came_from = {}
        g_score = { (cell.row, cell.col): float('inf') for row in self.grid for cell in row }
        f_score = { (cell.row, cell.col): float('inf') for row in self.grid for cell in row }

        start_pos = (self.start.row, self.start.col)
        end_pos = (self.end.row, self.end.col)

        g_score[start_pos] = 0
        f_score[start_pos] = self.heuristic(start_pos, end_pos)

        open_set.put((f_score[start_pos], next(self.counter), self.start))

        open_set_hash = {start_pos}

        while not open_set.empty():
            self.root.update()
            time.sleep(DELAY / 1000.0)

            current = open_set.get()[2]
            curr_pos = (current.row, current.col)
            open_set_hash.remove(curr_pos)

            if curr_pos == end_pos:
                self.reconstruct_path(came_from, current)
                return

            if current != self.start:
                current.set_state("visited")

            for neighbor in self.get_neighbors(current):
                n_pos = (neighbor.row, neighbor.col)
                temp_g = g_score[curr_pos] + 1

                if temp_g < g_score[n_pos]:
                    came_from[n_pos] = current
                    g_score[n_pos] = temp_g
                    f_score[n_pos] = temp_g + self.heuristic(n_pos, end_pos)
                    if n_pos not in open_set_hash:
                        open_set.put((f_score[n_pos], next(self.counter), neighbor))
                        open_set_hash.add(n_pos)

    def reconstruct_path(self, came_from, current):
        while (current.row, current.col) in came_from:
            current = came_from[(current.row, current.col)]
            if current != self.start:
                current.set_state("path")
            self.root.update()
            time.sleep(DELAY / 1000.0)

    def get_neighbors(self, cell):
        neighbors = []
        dirs = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for dr, dc in dirs:
            r, c = cell.row + dr, cell.col + dc
            if 0 <= r < self.size and 0 <= c < self.size:
                neighbor = self.grid[r][c]
                if neighbor.state != "wall":
                    neighbors.append(neighbor)
        return neighbors

    def heuristic(self, a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

def main():
    root = tk.Tk()
    root.title("A* Pathfinding Visualizer")

    size = simpledialog.askinteger("Grid Size", "Enter grid size (e.g., 10 for 10x10):", minvalue=5, maxvalue=50)
    if not size:
        return

    app = AStarVisualizer(root, size)
    root.mainloop()

if __name__ == "__main__":
    main()